In [1]:
from pathlib import Path

import polars as pl
import pandas as pd
import numpy as np

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import OrdinalEncoder

import lightgbm as lgb

In [2]:
exp_name = '000_stacking'

In [3]:
DATA_PATH = Path('/Users/gouyashuto/localrepository/atmacup18/input')
OUTPUT_DIR = Path('/Users/gouyashuto/localrepository/atmacup18/output')

In [4]:
train_df = pl.read_csv(DATA_PATH / 'train_features.csv')
test_df = pl.read_csv(DATA_PATH / 'test_features.csv')

print(train_df.shape)
print(test_df.shape)

(43371, 30)
(1727, 12)


In [5]:
CAT_COLS = ['gearShifter', 'scene']

TARGET_COLS = [
    'x_0', 'y_0', 'z_0',
    'x_1', 'y_1', 'z_1',
    'x_2', 'y_2', 'z_2',
    'x_3', 'y_3', 'z_3',
    'x_4', 'y_4', 'z_4',
    'x_5', 'y_5', 'z_5'
]

In [6]:
def get_agg_exprs(agg_cols) -> list[pl.Expr]:
    # 同一シーンから特徴量作成
    exprs = []
    exprs += [pl.col(agg_col).shift(-1).over("scene").alias(f"{agg_col}_shift-1") for agg_col in agg_cols] # 1ステップ前の時間の値
    exprs += [pl.col(agg_col).shift(1).over("scene").alias(f"{agg_col}_shift1") for agg_col in agg_cols] # 1ステップ後の時間の値
    exprs += [pl.col(agg_col).diff(-1).over("scene").alias(f"{agg_col}_diff-1") for agg_col in agg_cols] # 1ステップ前の時間の値との差分
    exprs += [pl.col(agg_col).diff(1).over("scene").alias(f"{agg_col}_diff1") for agg_col in agg_cols] # 1ステップ後の時間の値との差分
    exprs += [pl.col(agg_col).mean().over("scene").alias(f"{agg_col}_mean") for agg_col in agg_cols] # 同一シーンの平均値
    exprs += [pl.col(agg_col).std().over("scene").alias(f"{agg_col}_std") for agg_col in agg_cols] # 同一シーンの標準偏差
    exprs += [pl.col(agg_col).max().over("scene").alias(f"{agg_col}_max") for agg_col in agg_cols] # 同一シーンの最大値
    exprs += [pl.col(agg_col).min().over("scene").alias(f"{agg_col}_min") for agg_col in agg_cols] # 同一シーンの最小値
    return exprs

In [7]:
def preprocess(df: pl.DataFrame) -> pl.DataFrame:
    agg_cols = ['vEgo', 'aEgo', 'steeringAngleDeg', 'steeringTorque', 'gas'] # 同一シーンから集計する値のカラム名
    df = (
        df
        .with_columns(
            scene = pl.col('ID').str.split('_').list[0],
            decisecond = pl.col('ID').str.split('_').list[1].cast(pl.Int32),
        )
        .sort(['scene', 'decisecond'])
        .with_columns(get_agg_exprs(agg_cols))
    )
    return df

In [8]:
def train(X: pd.DataFrame, y: pd.DataFrame, target: str):
    lgb_params = {
        'objective': 'regression',
        'metric': 'mae',
        'learning_rate': 0.01,
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': 42,
        'n_jobs': -1,
    }
    gkf = GroupKFold(n_splits=5)
    groups = X['scene']

    models = []
    oof = np.zeros(len(X))
    # for fold, (train_idx, valid_idx) in enumerate(gkf.split(X, y[target], groups=groups)):
    for fold in range(5):
        print('=' * 10, f'fold: {fold} start' + '=' * 10)
        train_X = X.loc[X.fold != fold].drop(columns=['scene', 'fold'])
        train_y = y[target].loc[X.fold != fold]
        valid_X = X.loc[X.fold == fold].drop(columns=['scene', 'fold'])
        valid_y = y[target].loc[X.fold == fold]

        train_data = lgb.Dataset(train_X, train_y)
        valid_data = lgb.Dataset(valid_X, valid_y, reference=train_data)

        model = lgb.train(
            lgb_params,
            train_data,
            valid_sets=[valid_data],
            num_boost_round=10000,
            callbacks=[lgb.early_stopping(100)],
        )
        oof[X.fold == fold] = model.predict(valid_X)
        models.append(model)
    return models, oof

def predict(X: pd.DataFrame, models: list):
    preds = []
    for model in models:
        pred = model.predict(X.drop(columns=['scene']))
        preds.append(pred)
    return np.mean(preds, axis=0)

def evaluate(y_true: pd.DataFrame, y_pred: pd.DataFrame):
    return np.mean(np.abs(y_true - y_pred))

In [9]:
train_df = preprocess(train_df)

origin_test_ids = test_df['ID'].to_pandas()
test_df = preprocess(test_df)

print(train_df.shape)
print(test_df.shape)

(43371, 72)
(1727, 54)


In [10]:
train_df = train_df.to_pandas()
test_df = test_df.to_pandas()

In [11]:
# Sceneの名前を抜き出してDataFrameに加える
# train_df["scene"] = [i.split("_")[0] for i in train_df.ID]

# =======================================
# 5 Foldの作成
# =======================================

# train/val split
# SceneでFoldを分ける
scene_list = train_df.scene.unique()
fold_map = {}

for i, scene in enumerate(scene_list):
    fold_map[scene] = i % 5

train_df["fold"] = train_df.scene.map(fold_map)
train_df.head()

,ID,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,...,aEgo_max,steeringAngleDeg_max,steeringTorque_max,gas_max,vEgo_min,aEgo_min,steeringAngleDeg_min,steeringTorque_min,gas_min,fold
0,00066be8e20318869c38c66be466631a_320,5.701526,1.538456,-2.165777,-139.0,0.0,False,0.250,True,drive,...,1.538456,-2.165777,-44.0,0.250,5.701526,0.231099,-11.625697,-139.0,0.0,0
1,00066be8e20318869c38c66be466631a_420,11.176292,0.279881,-11.625697,-44.0,0.0,False,0.000,False,drive,...,1.538456,-2.165777,-44.0,0.250,5.701526,0.231099,-11.625697,-139.0,0.0,0
2,00066be8e20318869c38c66be466631a_520,10.472548,0.231099,-2.985105,-132.0,0.0,False,0.180,True,drive,...,1.538456,-2.165777,-44.0,0.250,5.701526,0.231099,-11.625697,-139.0,0.0,0
3,000fb056f97572d384bae4f5fc1e0f28_20,3.316744,1.276733,-31.725477,-114.0,0.0,False,0.255,True,drive,...,1.276733,7.632668,173.0,0.255,3.316744,-0.117775,-31.725477,-133.0,0.0,1
4,000fb056f97572d384bae4f5fc1e0f28_120,6.055565,-0.117775,7.632668,173.0,0.0,False,0.000,False,drive,...,1.276733,7.632668,173.0,0.255,3.316744,-0.117775,-31.725477,-133.0,0.0,1


In [12]:
predict_target_cols = []
for target in ['x', 'y', 'z']:
    for i in range(6):
        train_df[f"predict_{target}_{i}"] = 0
        test_df[f"predict_{target}_{i}"] = 0
        predict_target_cols.append(f"predict_{target}_{i}")

oof_dir = Path('/Users/gouyashuto/localrepository/atmacup18/output/baseline52/oof')

for i in range(5):
    train_df.loc[train_df.fold == i, predict_target_cols] = np.load(oof_dir / f'fold{i}_val_pred.npy')

test_df[predict_target_cols] = np.load(oof_dir.parent / 'test_preds.npy').mean(axis=0)

display(train_df.head())
display(test_df.head())

/var/folders/vq/4c8kgm65475ctdqgk6_bw_zw0000gn/T/ipykernel_31410/1437116301.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.73632812 4.984375   4.74609375 ... 4.92578125 4.6640625  4.0703125 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_df.loc[train_df.fold == i, predict_target_cols] = np.load(oof_dir / f'fold{i}_val_pred.npy')
/var/folders/vq/4c8kgm65475ctdqgk6_bw_zw0000gn/T/ipykernel_31410/1437116301.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.01679993 -0.01702881 -0.02026367 ... -0.00261688  0.02404785
  0.0491333 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_df.loc[train_df.fold == i, predict_target_cols] = np.load(oof_dir / f'fold{i}_val_pred.npy')
/var/folders/vq/4c8kgm65475ctdqgk6_bw_zw0000gn/

,ID,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,...,predict_y_2,predict_y_3,predict_y_4,predict_y_5,predict_z_0,predict_z_1,predict_z_2,predict_z_3,predict_z_4,predict_z_5
0,00066be8e20318869c38c66be466631a_320,5.701526,1.538456,-2.165777,-139.0,0.0,False,0.250,True,drive,...,0.119080,13.445312,-0.180176,0.170654,17.437500,-0.274170,0.247559,21.609375,-0.366699,0.309570
1,00066be8e20318869c38c66be466631a_420,11.176292,0.279881,-11.625697,-44.0,0.0,False,0.000,False,drive,...,-0.001527,21.593750,0.064880,-0.029175,27.093750,0.160400,-0.030014,32.500000,0.220825,-0.062866
2,00066be8e20318869c38c66be466631a_520,10.472548,0.231099,-2.985105,-132.0,0.0,False,0.180,True,drive,...,0.006664,20.968750,-0.312744,0.013977,26.421875,-0.476318,0.042175,31.921875,-0.647461,0.041534
3,000fb056f97572d384bae4f5fc1e0f28_20,3.316744,1.276733,-31.725477,-114.0,0.0,False,0.255,True,drive,...,-0.018555,11.250000,0.531250,-0.030075,13.992188,0.752930,-0.018341,16.765625,1.049805,-0.028473
4,000fb056f97572d384bae4f5fc1e0f28_120,6.055565,-0.117775,7.632668,173.0,0.0,False,0.000,False,drive,...,0.030365,8.835938,-0.446533,0.038300,11.609375,-0.596680,0.058685,14.554688,-0.725586,0.096375


,ID,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,...,predict_y_2,predict_y_3,predict_y_4,predict_y_5,predict_z_0,predict_z_1,predict_z_2,predict_z_3,predict_z_4,predict_z_5
0,012baccc145d400c896cb82065a93d42_120,3.374273,-0.019360,-34.008415,17.0,0.0,False,0.0,False,drive,...,0.004251,6.336719,0.040533,0.014632,7.878125,0.072980,0.019846,9.348437,0.122693,0.026746
1,012baccc145d400c896cb82065a93d42_220,2.441048,-0.022754,307.860077,295.0,0.0,True,0.0,False,drive,...,0.010564,3.069531,3.013672,0.023242,3.598438,4.293750,0.019300,4.112500,5.768750,0.038300
2,012baccc145d400c896cb82065a93d42_320,3.604152,-0.286239,10.774388,-110.0,0.0,True,0.0,False,drive,...,-0.023334,6.393750,-0.097316,-0.038749,7.826562,-0.164950,-0.051001,9.167188,-0.262469,-0.072753
3,012baccc145d400c896cb82065a93d42_420,2.048902,-0.537628,61.045235,189.0,0.0,True,0.0,False,drive,...,-0.034274,3.173828,1.094751,-0.057326,3.703516,1.642017,-0.087286,4.143359,2.276440,-0.123018
4,01d738e799d260a10f6324f78023b38f_120,2.201528,-1.898600,5.740093,-41.0,0.0,True,0.0,False,drive,...,-0.046704,2.311719,0.016077,-0.064423,2.382812,0.017225,-0.084155,2.349023,0.026129,-0.097980


In [13]:
remove_columns = ['ID']
X = train_df.drop(columns=remove_columns + TARGET_COLS)
y = train_df[TARGET_COLS]

test_X = test_df.drop(columns=remove_columns)

oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[CAT_COLS] = oe.fit_transform(X[CAT_COLS])
test_X[CAT_COLS] = oe.transform(test_X[CAT_COLS])

print(X.shape)
print(test_X.shape)

(43371, 72)
(1727, 71)


In [14]:
%%time

models_dict = {}
oof = pd.DataFrame(np.zeros_like(train_df.loc[:, TARGET_COLS]), columns=TARGET_COLS)

preds = test_df[['ID']]
preds[TARGET_COLS] = 0.0

for target in TARGET_COLS:
    print('=' * 10, f'target: {target} start' + '=' * 10)
    partial_model, partial_oofs = train(X, y, target)
    models_dict[target] = partial_model
    oof.loc[:, target] = partial_oofs
    preds.loc[:, target] = predict(test_X, models_dict[target])

========== target: x_0 start==========
========== fold: 0 start==========


<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1564]	valid_0's l1: 0.0614068
========== fold: 1 start==========
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[951]	valid_0's l1: 0.0620929
========== fold: 2 start==========
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[903]	valid_0's l1: 0.0610946
========== fold: 3 start==========
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1218]	valid_0's l1: 0.0632638
========== fold: 4 start==========
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[777]	valid_0's l1: 0.0626691
========== target: y_0 start==========
========== fold: 0 start==========
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[507]	valid_0's l1: 0.0314351
========== fold: 1 sta

In [15]:
for target in TARGET_COLS:
    print(f'{target} CV score: ', evaluate(y[target], oof[target]))
score = evaluate(y, oof)
print('Total CV score: ', score)

x_0 CV score:  0.062105030335105824
y_0 CV score:  0.03248770364722079
z_0 CV score:  0.025701465638211983
x_1 CV score:  0.1325317235760224
y_1 CV score:  0.07250823203733478
z_1 CV score:  0.05299822144373978
x_2 CV score:  0.22545671908336123
y_2 CV score:  0.12729037481698907
z_2 CV score:  0.0809430127655521
x_3 CV score:  0.34913832002046613
y_3 CV score:  0.20506128117010233
z_3 CV score:  0.10963126406336234
x_4 CV score:  0.5017995442976606
y_4 CV score:  0.3096934339821425
z_4 CV score:  0.13913533551419008
x_5 CV score:  0.6770215943221658
y_5 CV score:  0.44108702152944756
z_5 CV score:  0.1696468508705205
Total CV score:  0.2063465071729776


In [16]:
submission = pd.DataFrame(origin_test_ids).merge(preds, on='ID', how='left').drop(columns=['ID'])

output_path = OUTPUT_DIR / f'{exp_name}_{score:.4f}_submission.csv'
if output_path.exists():
    assert False, f'output file already exists. {output_path}'

submission.to_csv(output_path, index=False)
submission

AssertionError: output file already exists. /Users/gouyashuto/localrepository/atmacup18/output/000_stacking_0.2063_submission.csv